In [9]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook as tqdm
import pickle

In [ ]:
links = []
for page_num in tqdm(range(15000)):
    response = requests.get(f"https://kuruc.info/to/1/{page_num}0/")
    soup = BeautifulSoup(response.content, 'html.parser')
    titles = soup.find_all("div", class_="alcikkheader")
    links.extend(["https://kuruc.info" + title.find("a").get("href") for title in titles])

In [10]:
with open("links.p", "rb") as f:
    links = pickle.load(f)

In [11]:
len(links)

149990

In [18]:
kurucinfo_dict = {}
for url in tqdm(links):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        datetime = soup.find("span", {'itemprop': "datePublished"}).text
        body = soup.find('div', {'itemprop': 'articleBody'})
        related_urls = []
        if "MTI" in body.text or " mti " in body.text or "Mti" in body.text:
            related_urls.append("MTI")
        for link in body.find_all("a"):
            try:
                if "http" in link.get('href'):
                    related_urls.append(link.get('href'))
                else:
                    related_urls.append("https://kuruc.info/" +  link.get('href'))
            except:
                pass
        kurucinfo_dict[url] = {
                "related_links" : related_urls,
                "publish_date" : datetime
            }
    else:
        print(url)

In [17]:
link

<a>kijelentette</a>

In [20]:
len(kurucinfo_dict)

149961

In [21]:
with open('kurucinfo_dict.p', 'wb') as f:
    pickle.dump(kurucinfo_dict, f)